In [ ]:
%matplotlib inline
import nilearn

In [ ]:
# Let's keep our notebook clean, so it's a little more readable!
import warnings
warnings.filterwarnings('ignore')

## [Understanding neuroimaging data](http://nilearn.github.io/manipulating_images/input_output.html)

### Text files: phenotype or behavior

Phenotypic or behavioral data are often provided as text or CSV (Comma Separated Values) file. They can be loaded with the [pandas package](https://pandas.pydata.org/) but you may have to specify some options. Here, we'll specify the `sep` field, since our data is tab-delimited rather than comma-delimited.

For our dataset, let's load participant level information:

In [ ]:
import os
import pandas as pd

data_dir = '/home/Desktop/MAIN_tutorial/'
participants = 'participants.tsv'
phenotypic_data = pd.read_csv(os.path.join(data_dir, participants), sep='\t')
phenotypic.head()

### Nifti data

For volumetric data, nilearn works with data stored in the [Nifti structure](http://nipy.org/nibabel/nifti_images.html) (via the [nibabel package](http://nipy.org/nibabel/)).

The NifTi data structure (also used in Analyze files) is the standard way of sharing data in neuroimaging research. Three main components are:

  * data:	raw scans in form of a numpy array:  
    `data = img.get_data()`
  * affine:	returns the transformation matrix that maps from voxel indices of the `numpy` array to actual real-world     locations of the brain:  
    `affine = img.affine`
  * header:	low-level informations about the data (slice duration, etc.):  
  `header = img.header`

It is important to appreciate that the representation of MRI data we'll be using is a big 4D matrix representing (3D MRI + 1D for time), stored in a single Nifti file.

### Niimg-like objects

Nilearn functions take as input argument what we call "Niimg-like objects":

Niimg: A Niimg-like object can be one of the following:

  * A string with a file path to a Nifti image
  * An SpatialImage from `nibabel`, ie an object exposing get_data() method and affine attribute, typically a Nifti1Image from `nibabel`.

Niimg-4D: Similarly, some functions require 4D Nifti-like data, which we call Niimgs or Niimg-4D. Accepted input arguments are:

  * A path to a 4D Nifti image
  * List of paths to 3D Nifti images
  * 4D Nifti-like object
  * List of 3D Nifti-like objects

**Note:** If you provide a sequence of Nifti images, all of them must have the same affine !

## [Manipulating and looking at data](http://nilearn.github.io/auto_examples/plot_nilearn_101.html#sphx-glr-auto-examples-plot-nilearn-101-py)

There is a whole section of the [Nilearn documentation](http://nilearn.github.io/plotting/index.html#plotting) on making pretty plots for neuroimaging data ! But let's start with a simple one.

In [ ]:
# Let's use a Nifti file that is shipped with nilearn
from nilearn import datasets

# Note that the variable MNI152_FILE_PATH is just a path to a Nifti file
print('Path to MNI152 template: {}'.format(datasets.MNI152_FILE_PATH))

In the above, MNI152_FILE_PATH is nothing more than a string with a path pointing to a nifti image. You can replace it with a string pointing to a file on your disk. Note that it should be a 3D volume, and not a 4D volume.

In [ ]:
from nilearn import plotting
plotting.view_img(datasets.MNI152_FILE_PATH)

We can also directly manipulate these images using Nilearn ! As an example, let's try smoothing this image.

In [ ]:
from nilearn import image
smooth_anat_img = image.smooth_img(datasets.MNI152_FILE_PATH, fwhm=6)

# While we are giving a file name as input, the function returns
# an in-memory object:
print(smooth_anat_img)

In [ ]:
plotting.view_img(smooth_anat_img)

We can then save this manipulated image from in-memory to disk as follows:

In [ ]:
smooth_anat_img.to_filename('smooth_anat_img.nii.gz')
os.getcwd()  # We'll' check our "current working directory" (cwd) to see where the file was saved

## [Convert the fMRI volumes to a data matrix](http://nilearn.github.io/auto_examples/plot_decoding_tutorial.html#convert-the-fmri-volume-s-to-a-data-matrix)

These are some really lovely images, but for machine learning we want matrices ! Then we can use all of the techniques we learned this morning.

To transform our Nifti images into matrices, we'll use the `nilearn.input_data.NiftiMasker` to extract the fMRI data using a parcellation and convert it to data matrix.

In [ ]:
# First, let's load a parcellation that we'd like to use
multiscale = datasets.fetch_atlas_basc_multiscale_2015()
plotting.plot_roi(multiscale.scale064)

In [ ]:
from nilearn.input_data import NiftiLabelsMasker
masker = NiftiLabelsMasker(labels_img=multiscale.scale064, standardize=True)
masker

In [ ]:
# We give the masker a filename and retrieve a 2D array ready
# for machine learning with scikit-learn !
fmri_filename = 'downsampled_derivatives:fmriprep:sub-pixar001:sub-pixar001_task-pixar_run-001_swrf_bold.nii.gz'
fmri_matrix = masker.fit_transform(os.path.join(data_dir, fmri_filename))

In [ ]:
print(fmri_masked)